In [3]:
!pip install sentence_transformers
!pip install neuspell
!pip install ProText

     |████████████████████████████████| 184 kB 596 kB/s eta 0:00:01
     |████████████████████████████████| 123 kB 4.7 MB/s eta 0:00:01


In [32]:
import scipy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import models, SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased')

import neuspell # For spell correction
from neuspell import BertChecker

from ProText import gen, low, dig,stopw, lemma, stem, spell, clean_len, WCloud, countvec, tfidf # Text pre-Processing package

In [33]:
import pandas as pd

df = pd.read_csv("../input/flipkart-products/flipkart_com-ecommerce_sample.csv")

In [35]:
# Dropping all "Product Name" duplicates
df1 = df.copy()
df1.drop_duplicates(subset ="product_name", keep = "first", inplace = True)
df1.shape

(12676, 15)

In [36]:
# Concating and creating "keywords" columns
df1["keywords"] = df1['product_name'].astype('str')+" "+df1['description'].astype('str')+" "+df1['brand'].astype('str')

In [37]:
df1.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,keywords
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",Alisha Solid Women's Cycling Shorts Key Featur...
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",FabHomeDecor Fabric Double Sofa Bed FabHomeDec...
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",AW Bellies Key Features of AW Bellies Sandals ...
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",Sicons All Purpose Arnica Dog Shampoo Specific...
5,c2a17313954882c1dba461863e98adf2,2016-03-25 22:59:23 +0000,http://www.flipkart.com/eternal-gandhi-super-s...,Eternal Gandhi Super Series Crystal Paper Weig...,"[""Eternal Gandhi Super Series Crystal Paper We...",PWTEB7H2E4KCYUE3,430.0,430.0,"[""http://img5a.flixcart.com/image/paper-weight...",False,Key Features of Eternal Gandhi Super Series Cr...,No rating available,No rating available,Eternal Gandhi,"{""product_specification""=>[{""key""=>""Model Name...",Eternal Gandhi Super Series Crystal Paper Weig...


In [38]:
# Text pre-processing on '"keywords"
df1['keywords'] = df1['keywords'].apply(gen).apply(low).apply(stopw).apply(lemma)

In [39]:
corpus = df1.iloc[: , -1]

In [40]:
corpus

0        alisha solid woman cycling short key feature a...
1        fabhomedecor fabric double sofa bed fabhomedec...
2        aw belly key feature aw belly sandal wedge hee...
4        sicons purpose arnica dog shampoo specificatio...
5        eternal gandhi super series crystal paper weig...
                               ...                        
19936    purple woman heel flipkart com buy purple woma...
19948    uberlyfe large vinyl sticker buy uberlyfe larg...
19958    witch comfy hue woman wedge flipkart com buy w...
19962    stylistry woman heel flipkart com buy stylistr...
19976    uberlyfe extra large vinyl sticker uberlyfe ex...
Name: keywords, Length: 12676, dtype: object

In [41]:
# Creating corpus from data set
corpus_embeddings = model.encode(corpus.values)

Batches:   0%|          | 0/397 [00:00<?, ?it/s]

In [42]:
""" select spell checkers & load """
spellcheck = BertChecker()
spellcheck.from_pretrained("subwordbert-probwordnoise") # BERT

loading vocab from path:subwordbert-probwordnoise/vocab.pkl
initializing model


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SubwordBert(
  (bert_dropout): Dropout(p=0.2, inplace=False)
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): 

In [43]:
def find_similar(vector_representation, all_representations, k=1):
    similarity_matrix = cosine_similarity(vector_representation, all_representations)
    np.fill_diagonal(similarity_matrix, 0)
    similarities = similarity_matrix[0]
    if k == 1:
        return [np.argmax(similarities)]
    elif k is not None:
        return np.flip(similarities.argsort()[-k:][::1])

# Query for "women clothing"

In [54]:
X=input("Search Engine:")
queries = spellcheck.correct(X)
print(queries)

Search Engine: womns clothin


women clothing


In [62]:
query_embeddings = model.encode([queries])
K = 5 # no. of rows that has to be extracted
distilbert_similar_indexes = find_similar(query_embeddings, corpus_embeddings, K)
df1['product_name'].iloc[distilbert_similar_indexes]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

12481    Seeyaar Women's Striped Casual Shirt
12393      Seeyaar Women's Solid Casual Shirt
12468      Femella Women's Solid Casual Shirt
11924    Femella Women's Printed Casual Shirt
10075             Rasi Silks Women's Leggings
Name: product_name, dtype: object

# Query men clothing

In [63]:
X=input("Search Engine:")
queries = spellcheck.correct(X)
print(queries)

Search Engine: mens clothing


mens clothing


In [64]:
query_embeddings = model.encode([queries])
K = 5 # no. of rows that has to be extracted
distilbert_similar_indexes = find_similar(query_embeddings, corpus_embeddings, K)
df1['product_name'].iloc[distilbert_similar_indexes]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

788                       Wrangler Men's Casual Shirt
9874                Sloper Regular Fit Men's Trousers
438               Status Quo Men's Solid Casual Shirt
16665    Indian Weller Men's Woven Formal Linen Shirt
13858           Okane Striped Men's Polo Neck T-Shirt
Name: product_name, dtype: object

# Query "girl baby clothing"

In [66]:
X=input("Search Engine:")
queries = spellcheck.correct(X)
print(queries)

Search Engine: girl baby clothing


girl baby clothing


In [67]:
query_embeddings = model.encode([queries])
K = 5 # no. of rows that has to be extracted
distilbert_similar_indexes = find_similar(query_embeddings, corpus_embeddings, K)
df1['product_name'].iloc[distilbert_similar_indexes]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

18598                    Yo Baby Girl's Trousers
16585    kawal fashion Pyjama Baby Girl's  Combo
11212       RTD Baby Girl's Kurta and Pyjama Set
5468          Yashasvi Baby Girl's Layered Dress
8004      Beebay Baby Girl's Fit and Flare Dress
Name: product_name, dtype: object